##Creating the model

#Importing packages

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

keras = tf.keras

TensorFlow 2.x selected.


To have balanced training data we need to pick equal amount of examples of each class

In [0]:
from google.colab import drive
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
drive.mount('/content/gdrive')
os.chdir(r'/content/gdrive/My Drive')
#create training, test, eval datasets

def cr_training():
  df=pd.read_csv('./contest/images_labelling.csv')
  data_train=pd.DataFrame()

  for i in range(25):
    data_train=data_train.append(df[df['label']==i].copy(deep=False)[0:125])

  for x in data_train['boxid']:
    foto=Image.open('./contest/images/'+str(x)+'.png')
    foto = np.asarray( foto, dtype="float32")
    foto = (foto/127.5-1)
    image = tf.image.resize(foto, (83,45))
    label =data_train[data_train['boxid']==x]['label']
    yield (image,label)

def cr_test():
  df=pd.read_csv('./contest/images_labelling.csv')
  data_test=pd.DataFrame()

  for i in range(25):
    data_test=data_test.append(df[df['label']==i].copy(deep=False)[130:150])

  for x in data_test['boxid']:
    foto=Image.open('./contest/images/'+str(x)+'.png')
    foto = np.asarray( foto, dtype="float32")
    foto = (foto/127.5-1)
    image = tf.image.resize(foto, (83,45))
    label =data_test[data_test['boxid']==x]['label']
    yield (image,label)

def cr_eval():
  df=pd.read_csv('./contest/images_labelling.csv')
  data_eval=pd.DataFrame()

  for i in range(25):
    data_eval=data_eval.append(df[df['label']==i].copy(deep=False)[125:130])

  for x in data_eval['boxid']:
    foto=Image.open('./contest/images/'+str(x)+'.png')
    foto = np.asarray( foto, dtype="float32")
    foto = (foto/127.5-1)
    image = tf.image.resize(foto, (83,45))
    label =data_eval[data_eval['boxid']==x]['label']
    yield (image,label)
train_dataset=tf.data.Dataset.from_generator(cr_training, output_types=(tf.float32, tf.int32),output_shapes=(tf.TensorShape([83,45,3]), tf.TensorShape([None])))
test_dataset=tf.data.Dataset.from_generator(cr_test, output_types=(tf.float32, tf.int32),output_shapes=(tf.TensorShape([83,45,3]), tf.TensorShape([None])))
eval_dataset=tf.data.Dataset.from_generator(cr_eval, output_types=(tf.float32, tf.int32),output_shapes=(tf.TensorShape([83,45,3]), tf.TensorShape([None])))

#define target variable as one hot, also transform it to int to reduce memory needed
def format_example(image, label):
  label = tf.one_hot(label, 25)
  label = tf.cast(label, tf.int32)
  return image,label

train_dataset.map(format_example)
test_dataset.map(format_example)
eval_dataset.map(format_example)
#shuffle the data
BATCH_SIZE =20
SHUFFLE_BUFFER_SIZE = 1000
train_batches = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = eval_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_batches = test_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##Conctruct the model

In [0]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu',
                 input_shape=(83, 45 ,3)),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32,(3, 3) , activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.25),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(25,activation='softmax')
])


"model = Sequential()\nmodel.add(Conv2D(32, (3, 3), padding='same',\n                 input_shape=(83, 45 ,3)))\nmodel.add(Activation('relu'))\nmodel.add(Conv2D(32, (3, 3)))\nmodel.add(Activation('relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\nmodel.add(Dropout(0.25))\n\nmodel.add(Conv2D(64, (3, 3), padding='same'))\nmodel.add(Activation('relu'))\nmodel.add(Conv2D(64, (3, 3)))\nmodel.add(Activation('relu'))\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\nmodel.add(Dropout(0.25))\n\nmodel.add(Flatten())\nmodel.add(Dense(512))\nmodel.add(Activation('relu'))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(25))\nmodel.add(Activation('softmax'))"

#Set the optimizer, loss, and train the model

In [0]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_batches,
                    epochs=150,
                    validation_data=test_batches)

Epoch 1/150
157/157 [==============================] - 1025s 7s/step - loss: 2.3279 - accuracy: 0.3722 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/150
157/157 [==============================] - 11s 71ms/step - loss: 1.2692 - accuracy: 0.6282 - val_loss: 1.4297 - val_accuracy: 0.5700
Epoch 3/150
157/157 [==============================] - 11s 71ms/step - loss: 0.9445 - accuracy: 0.7366 - val_loss: 1.0912 - val_accuracy: 0.6740
Epoch 4/150
157/157 [==============================] - 11s 71ms/step - loss: 0.7593 - accuracy: 0.8042 - val_loss: 1.0379 - val_accuracy: 0.7420
Epoch 5/150
157/157 [==============================] - 11s 70ms/step - loss: 0.6648 - accuracy: 0.8374 - val_loss: 0.8693 - val_accuracy: 0.7640
Epoch 6/150
157/157 [==============================] - 11s 70ms/step - loss: 0.5625 - accuracy: 0.8656 - val_loss: 0.9943 - val_accuracy: 0.7740
Epoch 7/150
157/157 [==============================] - 11s 71ms/step - loss: 0.5178 - accuracy: 0.8726 - val_loss: 0.8815 

#Evaluate model with previously unseen data, and get approx. 90% accuracy

In [0]:
scores=model.evaluate(validation_batches)

7/7 [==============================] - 35s 5s/step - loss: 0.5413 - accuracy: 0.8800


#Save the model to deploy it

In [0]:
model.save('my_model.h5') 